<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/cs_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mz2j6faj
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-mz2j6faj
  Resolved https://github.com/huggingface/transformers to commit 06343b06335a1f8417bd32d3ffc7cf2cca9a24ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [26]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [ ]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [27]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "cs", split="test")
#column_name = "audio, normalized_text"

In [28]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [29]:
model = whisper.load_model("base")

In [30]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [31]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="cs")
    return result["text"]

In [32]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [33]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score


In [34]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
count = 0

for sample in voxpopuli_data:
    audio_file_path = sample['audio']['path']
    ground_truth_text = sample['normalized_text']

    # Transcribe the audio
    transcription = transcribe_audio(audio_file_path)

    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    count += 1

    print(f"Processed {count} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")
    # Calculate average metrics
average_wer = total_wer / count
average_precision = total_precision / count
average_recall = total_recall / count
average_f1 = total_f1 / count

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")


Processed 1 files, WER: 0.5806451612903226, Precision: 0.4444444444444444, Recall: 0.41379310344827586, F1: 0.4285714285714286
Processed 2 files, WER: 0.6363636363636364, Precision: 0.36363636363636365, Recall: 0.4444444444444444, F1: 0.39999999999999997
Processed 3 files, WER: 0.7741935483870968, Precision: 0.2857142857142857, Recall: 0.27586206896551724, F1: 0.28070175438596495
Processed 4 files, WER: 0.6590909090909091, Precision: 0.41025641025641024, Recall: 0.4444444444444444, F1: 0.42666666666666664
Processed 5 files, WER: 0.782608695652174, Precision: 0.45454545454545453, Recall: 0.21739130434782608, F1: 0.29411764705882354
Processed 6 files, WER: 0.8181818181818182, Precision: 0.25, Recall: 0.21875, F1: 0.23333333333333334
Processed 7 files, WER: 0.6538461538461539, Precision: 0.4166666666666667, Recall: 0.4166666666666667, F1: 0.4166666666666667
Processed 8 files, WER: 0.6363636363636364, Precision: 0.36363636363636365, Recall: 0.36363636363636365, F1: 0.36363636363636365
Proc